# Menambahkan Dataset
## https://www.kaggle.com/datasets/miadul/hypertension-risk-prediction-dataset/data

In [33]:
!pip install langchain_community
!pip install replicate

In [34]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

# Model setup
output = Replicate(
    model = "ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=api_token,
)

In [35]:
# Impor modul
import os
import json
import pandas as pd
from google.colab import userdata
from langchain_community.llms import Replicate

from google.colab import drive
drive.mount('/content/drive')

# Baca CSV langsung dari Google Drive
file_path = '/content/drive/MyDrive/Capstone Project_Hacktiv8_IBM/hypertension_dataset.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# Ambil beberapa data sebagai contoh (misal 3 pasien)
samples = df.sample(3, random_state=42)

# Loop tiap baris → kirim ke model
for idx, row in samples.iterrows():
    input_data = row.drop("Has_Hypertension").to_dict()
    input_json = json.dumps(input_data, indent=2)

    prompt = f"""
### Instruction:
Berdasarkan data pasien berikut, prediksi apakah pasien memiliki hipertensi (Yes/No), dan jelaskan alasannya secara singkat berdasarkan fitur yang tersedia.

### Input:
{input_json}

### Response:
"""

# Kirim prompt ke model
print(f"🧪 Prompt untuk pasien {idx}:\n")
result = output.invoke(prompt)
print(result)
print("-" * 80)

🧪 Prompt untuk pasien 1052:

Prediksi: Yes

Alasan:

1. Age: The patient is 60 years old, which increases the risk of hypertension due to age-related arterial stiffening.

2. Salt Intake: With a salt intake of 8.6 grams, this is higher than the recommended daily limit (5g for adults), contributing to increased blood pressure.

3. Stress Score: A moderate stress score of 4 indicates that the patient experiences stress, which can elevate blood pressure levels.

4. BP History: The patient has a history of hypertension, suggesting that they are prone to high blood pressure.

5. Sleep Duration: A sleep duration of 6.5 hours is below the recommended 7-9 hours for adults. Insufficient sleep can lead to increased stress hormones and higher blood pressure.

6. BMI: A BMI of 22.7 indicates that the patient is within the normal weight range, which generally reduces the risk of hypertension. However, this feature alone is not sufficient to counterbalance the other risk factors present in this case

In [37]:
prompt = f"""
## 🩺 Instruction:
Berdasarkan data pasien berikut, prediksikan apakah pasien memiliki hipertensi (Jawaban: **Yes** atau **No**).
Berikan penjelasan singkat berdasarkan fitur-fitur yang tersedia.

## 📄 Data Pasien:
{input_json}

## 🧠 Response:
"""

In [43]:
pars = {
    "top_k": 50,
    "top_p": 0.95,
    "max_new_tokens": 300,
    "min_tokens": 50,
    "random_seed": None,
    "repetition_penalty": 1.2,
    "stopping_criteria": "length (256 tokens)",
    "stop_sequences": None
}

In [44]:
print(prompt)


## 🩺 Instruction:
Berdasarkan data pasien berikut, prediksikan apakah pasien memiliki hipertensi (Jawaban: **Yes** atau **No**). 
Berikan penjelasan singkat berdasarkan fitur-fitur yang tersedia.

## 📄 Data Pasien:
{
  "Age": 60,
  "Salt_Intake": 8.6,
  "Stress_Score": 4,
  "BP_History": "Hypertension",
  "Sleep_Duration": 6.5,
  "BMI": 22.7,
  "Medication": "Diuretic",
  "Family_History": "Yes",
  "Exercise_Level": "Low",
  "Smoking_Status": "Non-Smoker"
}

## 🧠 Response:



In [45]:
response = output.invoke(prompt, parameters=pars)
print(response)

Yes

### Explanation:

1. **Age**: The patient is 60 years old. Age is a risk factor for hypertension.

2. **Salt Intake**: With a salt intake of 8.6 grams, this is above the WHO recommended maximum of 5 grams per day, which can contribute to hypertension.

3. **Stress Score**: A score of 4 indicates moderate stress levels. Chronic stress can elevate blood pressure.

4. **BP History**: The patient has a history of hypertension, suggesting a predisposition to the condition.

5. **Sleep Duration**: Sleeping for 6.5 hours, although within the recommended 7-9 hours, does not indicate poor sleep quality which can be linked to hypertension.

6. **BMI**: A BMI of 22.7 is within the healthy range (18.5-24.9), suggesting the patient is not overweight, which can be a risk factor for hypertension.

7. **Medication**: Currently taking a diuretic, a common medication for managing hypertension, indicating the patient's current condition is being actively treated.

8. **Family History**: Having a fam